<a href="https://colab.research.google.com/github/greengerong/awesome-llm/blob/main/colab/sd3.5/SD3_5Large_InstantX_IPA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers diffusers accelerate omegaconf bitsandbytes
%cd /content/
!git lfs install
!git lfs clone https://huggingface.co/InstantX/SD3.5-Large-IP-Adapter

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 3.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 6.9 MB/s eta 0:00:00
  Created wheel for antlr4-python3-runtime: filename=antlr4_python3_runtime-4.9.3-py3-none-any.whl size=144555 sha256=51b7f14b23f25535916f375da768a498067b9fddc8ca06277b0ea79dc19b151c
  Stored in directory: /root/.cache/pip/wheels/12/93/dd/1f6a127edc45659556564c5730f6d4e300888f4bca2d4c5a88
Successfully built antlr4-python3-runtime


/content
Git LFS initialized.
          with new flags from 'git clone'

'git clone' has been updated in upstream Git to have comparable
speeds to 'git lfs clone'.
Cloning into 'SD3.5-Large-IP-Adapter'...
remote: Enumerating objects: 30, done.
remote: Counting objects: 100% (26/26), done.
remote: Compressing objects: 100% (25/25), done.
remote: Total 30 (delta 5), reused 0 (delta 0), pack-reused 4 (from 1)
Unpacking objects: 100% (30/30), 1.58 MiB | 6.91 MiB/s, done.


In [7]:
!pip install huggingface_hub
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) n
Token is valid (permission: read).
The token `colab` has been saved to /root/.cache/huggingface/stored_tokens
Your token has been saved to /root/.cache/huggingface/token
Login successful.
The current active token is: `colab`


In [1]:
%cd /content/SD3.5-Large-IP-Adapter

import torch
from PIL import Image
from diffusers import BitsAndBytesConfig, SD3Transformer2DModel
from models.transformer_sd3 import SD3Transformer2DModel
from pipeline_stable_diffusion_3_ipa import StableDiffusion3Pipeline

nf4_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)


def load_model(model_path, ip_adapter_path, image_encoder_path):
  model_nf4 = SD3Transformer2DModel.from_pretrained(model_path, subfolder="transformer", torch_dtype=torch.bfloat16, quantization_config=nf4_config)
  pipe = StableDiffusion3Pipeline.from_pretrained(model_path, transformer=model_nf4, torch_dtype=torch.bfloat16,).to("cuda")
  pipe.init_ipadapter(ip_adapter_path=ip_adapter_path, image_encoder_path=image_encoder_path, nb_token=64,)
  return pipe


def generate_images(pipe, image_info_list, output_dir):
    for i, (image_path, prompt) in enumerate(image_info_list):
        ref_img = Image.open(image_path).convert('RGB')


        # Generate image using the model
        generated_image = pipe(
            width=1024,
            height=1024,
            prompt=prompt,
            negative_prompt="lowres, low quality, worst quality",
            num_inference_steps=24,
            guidance_scale=5.0,
            generator=torch.Generator("cuda").manual_seed(42),
            clip_image=ref_img,
            ipadapter_scale=0.5,
        ).images[0]

        output_path = f"{output_dir}/result_{i}.jpg"
        generated_image.save(output_path)
        print(f"Generated image saved to: {output_path}")


if __name__ == "__main__":
    model_path = 'stabilityai/stable-diffusion-3.5-large'
    ip_adapter_path = './ip-adapter.bin'
    image_encoder_path = "google/siglip-so400m-patch14-384"

    # Load the model
    pipe = load_model(model_path, ip_adapter_path, image_encoder_path)


    # Define a list of tuples with (image_path, prompt)
    image_info_list = [
        ('./assets/1.jpg', 'a cat'),
        ('./assets/1.jpg', 'a dog'),
        ('./assets/1.jpg', 'a Asian women,20-old-years, black suit, office'),
        # Add more (image, prompt) pairs as needed
    ]

    output_directory = './results'
    generate_images(pipe, image_info_list, output_directory)

/content/SD3.5-Large-IP-Adapter


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

Loading pipeline components...:   0%|          | 0/9 [00:00<?, ?it/s]

You set `add_prefix_space`. The tokenizer needs to be converted from the slow tokenizers


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

OutOfMemoryError: CUDA out of memory. Tried to allocate 80.00 MiB. GPU 0 has a total capacity of 14.75 GiB of which 21.06 MiB is free. Process 105477 has 14.72 GiB memory in use. Of the allocated memory 14.53 GiB is allocated by PyTorch, and 79.25 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)